# DAILY GLOBAL HISTORICAL CLIMATOLOGY NETWORK (GHCN-DAILY) 

Version 3.24

[Andreas Loukas](https://andreasloukas.wordpress.com/), [EPFL LTS2](https://lts2.epfl.ch/), [Michaël Defferrard](http://deff.ch)

Menne, M.J., I. Durre, B. Korzeniewski, S. McNeal, K. Thomas, X. Yin, S. Anthony, R. Ray, 
R.S. Vose, B.E.Gleason, and T.G. Houston, 2012: Global Historical Climatology Network - 
Daily (GHCN-Daily), Version 3. 

The five core features are:

* PRCP = Precipitation (tenths of mm)
* SNOW = Snowfall (mm)
* SNWD = Snow depth (mm)
* TMAX = Maximum temperature (tenths of degrees C)
* TMIN = Minimum temperature (tenths of degrees C)

The other features are:

* ACMC = Average cloudiness midnight to midnight from 30-second ceilometer data (percent)
* ACMH = Average cloudiness midnight to midnight from manual observations (percent)
* ACSC = Average cloudiness sunrise to sunset from 30-second ceilometer data (percent)
* ACSH = Average cloudiness sunrise to sunset from manual observations (percent)
* AWDR = Average daily wind direction (degrees)
* AWND = Average daily wind speed (tenths of meters per second)
* DAEV = Number of days included in the multiday evaporation total (MDEV)
* DAPR = Number of days included in the multiday precipiation total (MDPR)
* DASF = Number of days included in the multiday snowfall total (MDSF)		  
* DATN = Number of days included in the multiday minimum temperature (MDTN)
* DATX = Number of days included in the multiday maximum temperature (MDTX)
* DAWM = Number of days included in the multiday wind movement (MDWM)
* DWPR = Number of days with non-zero precipitation included in multiday precipitation total (MDPR)
* EVAP = Evaporation of water from evaporation pan (tenths of mm)
* FMTM = Time of fastest mile or fastest 1-minute wind (hours and minutes, i.e., HHMM)
* FRGB = Base of frozen ground layer (cm)
* FRGT = Top of frozen ground layer (cm)
* FRTH = Thickness of frozen ground layer (cm)
* GAHT = Difference between river and gauge height (cm)
* MDEV = Multiday evaporation total (tenths of mm; use with DAEV)
* MDPR = Multiday precipitation total (tenths of mm; use with DAPR and DWPR, if available)
* MDSF = Multiday snowfall total 
* MDTN = Multiday minimum temperature (tenths of degrees C; use with DATN)
* MDTX = Multiday maximum temperature (tenths of degress C; use with DATX)
* MDWM = Multiday wind movement (km)
* MNPN = Daily minimum temperature of water in an evaporation pan (tenths of degrees C)
* MXPN = Daily maximum temperature of water in an evaporation pan (tenths of degrees C)
* PGTM = Peak gust time (hours and minutes, i.e., HHMM)
* PSUN = Daily percent of possible sunshine (percent)
* SN*# = Minimum soil temperature (tenths of degrees C) where * corresponds to a code for ground cover and # corresponds to a code for soil depth. For ground cover and depth codes see [readme](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt)
* SX*# = Maximum soil temperature (tenths of degrees C)  where * corresponds to a code for ground cover  and # corresponds to a code for soil depth.  See SN*# for ground cover and depth codes. 
* TAVG = Average temperature (tenths of degrees C) [Note that TAVG from source 'S' corresponds to an average for the period ending at 2400 UTC rather than local midnight]
* THIC = Thickness of ice on water (tenths of mm)	
* TOBS = Temperature at the time of observation (tenths of degrees C)
* TSUN = Daily total sunshine (minutes)
* WDF1 = Direction of fastest 1-minute wind (degrees)
* WDF2 = Direction of fastest 2-minute wind (degrees)
* WDF5 = Direction of fastest 5-second wind (degrees)
* WDFG = Direction of peak wind gust (degrees)
* WDFI = Direction of highest instantaneous wind (degrees)
* WDFM = Fastest mile wind direction (degrees)
* WDMV = 24-hour wind movement (km)	   
* WESD = Water equivalent of snow on the ground (tenths of mm)
* WESF = Water equivalent of snowfall (tenths of mm)
* WSF1 = Fastest 1-minute wind speed (tenths of meters per second)
* WSF2 = Fastest 2-minute wind speed (tenths of meters per second)
* WSF5 = Fastest 5-second wind speed (tenths of meters per second)
* WSFG = Peak gust wind speed (tenths of meters per second)
* WSFI = Highest instantaneous wind speed (tenths of meters per second)
* WSFM = Fastest mile wind speed (tenths of meters per second)
* WT** = Weather Type. For numeric codes ** see [readme](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt)
* WV** = Weather in the Vicinity. For numeric codes ** see [readme](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt)

[FTP link](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import shutil
import sys
sys.path.append('../..')
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # change to chosen GPU to use, nothing if work on CPU

import numpy as np
import time
import matplotlib.pyplot as plt
import healpy as hp
import pandas as pd

from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
from deepsphere import models, experiment_helper, plot, utils
from deepsphere.data import LabeledDatasetWithNoise, LabeledDataset
import hyperparameters

In [ ]:
datapath = "/mnt/nas/LTS2/datasets/ghcn-daily/processed/"
rawpath = "/mnt/nas/LTS2/datasets/ghcn-daily/raw/"
newdatapath = "./data/ghcn-daily/processed/"

In [ ]:
years = np.arange(2010,2015)
years2 = np.arange(2010,2018)

feature_names = ['PRCP', 'TMIN', 'TMAX', 'SNOW']
n_features = len(feature_names)
n_years  = len(years)

## Station

In [ ]:
filename = 'stations_{:4d}-{:4d}.npz'.format(years[0], years[-1])

# only recompute if necessary
if not os.path.isfile(datapath+filename):
    
    # Variable   Columns   Type
    # ID            1-11   Character
    # LATITUDE     13-20   Real
    # LONGITUDE    22-30   Real
    # ELEVATION    32-37   Real
    # STATE        39-40   Character
    # NAME         42-71   Character
    # GSN FLAG     73-75   Character
    # HCN/CRN FLAG 77-79   Character
    # WMO ID       81-85   Character

    id_ghcn, lat, lon, elev, name = [], [], [], [], []
    with open(rawpath+'ghcnd-stations.txt', 'r') as f:
        for line in f:

            iid, ilat, ilon, ielev, iname = line[0:11], line[12:20], line[21:30], line[31:37], line[41:71]         
            # state, GSN_flag, HCNCRN_flag, WMO_id = line[38:40], line[72:75], line[76:79], line[80:85] 

            assert (not iid.isspace()) and (not ilat.isspace()) and (not ilon.isspace()) \
                and (not ielev.isspace()) and (not iname.isspace())

            id_ghcn.append(iid.strip())
            lat.append(float(ilat.strip()))
            lon.append(float(ilon.strip()))
            elev.append(float(ielev.strip()))
            name.append(iname.strip())

    id_ghcn, lat, lon, elev, name = np.array(id_ghcn), np.array(lat), np.array(lon), np.array(elev), np.array(name)

    # Identify *relevant* stations: These are stations giving measurements in the years of interest
    # that also have known coordinates.

    # first, construct the set of all ghcn identifiers encountered in daily
    id_ghcn_relevant = set([])

    for yearIdx,year in enumerate(years):

        filename2 = rawpath+'{:4}.csv.gz'.format(year)
        print('- pre-parsing : {}'.format(filename2))

        df = pd.read_csv(filename2, names=['id_ghcn', 'date', 'type', 'value', '?0', '?1', '?2', '?3'], \
                         nrows=None, usecols=[0,1,2,3])


        id_ghcn_relevant |= set(df["id_ghcn"].values)

    # second, find identifiers both in id_ghcn and id_ghcn_relevant
    id_ghcn_relevant = set(id_ghcn) & id_ghcn_relevant

    # third, keep only relevant station data 
    keep = [id in id_ghcn_relevant for id in id_ghcn] 
    id_ghcn, lat, lon, elev, name = id_ghcn[keep], lat[keep], lon[keep], elev[keep], name[keep] 

    # free up some memory
    del id_ghcn_relevant, keep

    np.savez_compressed(newdatapath+filename, id_ghcn=id_ghcn, lat=lat, lon=lon, elev=elev, name=name, years=years)
    
else:
    station_file = np.load(datapath+filename)
    id_ghcn, lat, lon, elev, name = station_file['id_ghcn'], station_file['lat'], station_file['lon'], station_file['elev'], station_file['name']
    del station_file
    
n_stations = id_ghcn.shape[0]
print('{} weather stations identified.'.format(n_stations))

#  a dictionary mapping GHCN ids to local ids (rows in id array) 
ghcn_to_local = dict(zip(id_ghcn, np.arange(n_stations)))

## Data

In [ ]:
feature_names = ['PRCP', 'TMIN', 'TMAX', 'SNOW', 'SNWD', 'AWND', 'WT']
# max stations, 50'469
# in 2010, in nbr stations [36k, 16k, 16k, 6k, 22k, 20k, 0, 0, 28, 1k, 6, 5k, 0]
# in 2011, in nbr stations [36k, 16k, 16k, 6k, 22k, 18k, 0, 0, 38, 1k, 22, 5k, 0]
# in 2014, in nbr stations [39k, 15k, 15k, 6k, 24k, 19k, 0, 0, 40, 1k, 49, 4k, 0]

filenames = []
datas = []
valid_days_list = []
n_stations_list = []
for feature_name in feature_names:
    filenames.append('data_{:4d}-{:4d}_{}.npz'.format(years[0], years[-1], feature_name))
    print(f'- Checking if file {filenames[-1]} exists..')

    # only recompute if necessary
    if not os.path.isfile(newdatapath+filenames[-1]):

        print('- The file is not there. Parsing everything from raw. This will take a while.')
        os.makedirs(newdatapath, exist_ok=True)
        # Load the station measurements into a year-list of dataframes
        df_years = []

        for yearIdx,year in enumerate(years):

            filename_year = rawpath+'{:4}.csv.gz'.format(year)
            print(' - parsing *{}*'.format(filename_year))

            df = pd.read_csv(filename_year, names=['id_ghcn', 'date', 'type', 'value', 'MF', 'qualityF', 'source', '?0'], \
                             nrows=None, usecols=[0,1,2,3,5])

            # create a new column with the id_local
            id_local = [ghcn_to_local.get(id_g) for id_g in df["id_ghcn"].values]
            id_local = [-1 if v is None else v for v in id_local]
            id_local = np.array(id_local).astype(np.int)

            df = df.assign(id_local=pd.Series(id_local, index=df.index).values)

            # remove measurement of stations with unknown id_local
            df = df[df.id_local != -1] 
            
            # replace measurements with bad quality flag
            #df.value[~df.qualityF.isna()] = np.nan
            df = df[df.qualityF.isna()]
            df = df.drop('qualityF', axis=1)
            
            df_years.append(df)

        del df, id_local
        print('done!')

        # Construct one array per feature and save it to disk

        # indicate for which days we have measurements (this also deals with months of different lengths)
        valid_days = np.zeros((n_years, 12, 31), dtype=np.bool)

        for _, name in enumerate(feature_names):

            print(f' - Looking at {name}')

            data = np.zeros((n_stations, n_years, 12, 31), dtype=np.float) * np.nan

            for yearIdx,year in enumerate(years):

                df = df_years[yearIdx]
                idf = df.loc[df.type.str.contains(name)]

                print(f'  - year {year}')

                # remove measurement of stations with unknown id_local
                idf = idf[idf.id_local != -1] 

                for monthIdx,month in enumerate(range(1,12+1)): 
                    for dayIdx,day in enumerate(range(1,31+1)):        

                        date = int('{:4d}{:02d}{:02d}'.format(year,month,day))
                        jdf = idf.loc[idf['date'] == date]

                        # sort data according to the id_local 
                        jdf.set_index('id_local', inplace=True)
                        jdf = jdf.sort_index()

                        index = jdf.index.values
                        if name is 'WT' or name is 'WV':
                            values = jdf.type.str.extract(r'(\d+)').values.astype(int)
                            values = values[:,0]
                        else:
                            values = jdf['value'].values.astype(np.float)

                        if len(index) != 0: 
                            data[index,yearIdx,monthIdx,dayIdx] = values
                            valid_days[yearIdx,monthIdx,dayIdx] = True

            print('  - saving to disk')
            np.savez_compressed(newdatapath+'data_{:4d}-{:4d}_{}.npz'.format(years[0], years[-1], name), data=data, valid_days=valid_days)

            del index, values, df, idf, jdf    

    else:    
        print('- Loading data from disk..')

        data_file = np.load(newdatapath+filenames[-1])
        data, valid_days = data_file['data'], data_file['valid_days']        
        n_stations = data.shape[0]
        print(f'- {n_stations} stations loaded.')
        data = data.reshape((n_stations, n_years*12*31))
        if feature_name == 'TMIN' or feature_name == 'TMAX' or feature_name == 'PRCP':
            data = data.astype(np.float)
            data /= 10
        datas.append(data)
        valid_days = np.squeeze(valid_days.reshape(n_years*12*31)).astype(np.bool)
        valid_days_list.append(valid_days)
        n_stations_list.append(n_stations)

In [ ]:
#feature_name = 'TMAX'
# feature_names
filenames = []
datas_old = []
valid_days_list = []
n_stations_list = []
for feature_name in feature_names[:4]:
    filenames.append('data_{:4d}-{:4d}_{}.npz'.format(years[0], years[-1], feature_name))
    print(f'- Checking if file {filenames[-1]} exists..')

    # only recompute if necessary
    if not os.path.isfile(datapath+filenames[-1]):

        print('- The file is not there. Parsing everything from raw. This will take a while.')
        # Load the station measurements into a year-list of dataframes
        df_years = []

        for yearIdx,year in enumerate(years):

            filename_year = rawpath+'{:4}.csv.gz'.format(year)
            print(' - parsing *{}*'.format(filename_year))

            df = pd.read_csv(filename_year, names=['id_ghcn', 'date', 'type', 'value', '?0', '?1', '?2', '?3'], \
                             nrows=None, usecols=[0,1,2,3])

            # create a new column with the id_local
            id_local = [ghcn_to_local.get(id_g) for id_g in df["id_ghcn"].values]
            id_local = [-1 if v is None else v for v in id_local]
            id_local = np.array(id_local).astype(np.int)

            df = df.assign(id_local=pd.Series(id_local, index=df.index).values)

            # remove measurement of stations with unknown id_local
            df = df[df.id_local != -1] 

            df_years.append(df)

        del df, id_local
        print('done!')

        # Construct one array per feature and save it to disk

        # indicate for which days we have measurements (this also deals with months of different lengths)
        valid_days = np.zeros((n_years, 12, 31), dtype=np.bool)

        for _, name in enumerate(feature_names):

            print(f' - Looking at {name}')

            data = np.zeros((n_stations, n_years, 12, 31), dtype=np.float) * np.nan

            for yearIdx,year in enumerate(years):

                df = df_years[yearIdx]
                idf = df.loc[df['type'] == name]

                print(f'  - year {year}')

                # remove measurement of stations with unknown id_local
                idf = idf[idf.id_local != -1] 

                for monthIdx,month in enumerate(range(1,12+1)): 
                    for dayIdx,day in enumerate(range(1,31+1)):        

                        date = int('{:4d}{:02d}{:02d}'.format(year,month,day))
                        jdf = idf.loc[idf['date'] == date]

                        # sort data according to the id_local 
                        jdf.set_index('id_local', inplace=True)
                        jdf = jdf.sort_index()

                        index = jdf.index.values
                        values = jdf['value'].values.astype(np.float)

                        if len(index) != 0: 
                            data[index,yearIdx,monthIdx,dayIdx] = values
                            valid_days[yearIdx,monthIdx,dayIdx] = True

            print('  - saving to disk')
            np.savez_compressed('processed/data_{:4d}-{:4d}_{}.npz'.format(years[0], years[-1], name), data=data, valid_days=valid_days)

            del index, values, df, idf, jdf    

    else:    
        print('- Loading data from disk..')

        data_file = np.load(datapath+filenames[-1])
        data, valid_days = data_file['data'], data_file['valid_days']        
        n_stations = data.shape[0]
        print(f'- {n_stations} stations loaded.')
        data = data.reshape((n_stations, n_years*12*31))
        if feature_name == 'TMIN' or feature_name == 'TMAX' or feature_name == 'PRCP':
            data = data.astype(np.float)
            data /= 10
        datas_old.append(data)
        valid_days = np.squeeze(valid_days.reshape(n_years*12*31)).astype(np.bool)
        valid_days_list.append(valid_days)
        n_stations_list.append(n_stations)

In [ ]:
plt.scatter(full_data[:,:,1], full_data[:,:,2], s=2)
plt.plot(range(-40, 41), range(-40, 41), 'r')

In [ ]:
plt.scatter(datas[1][:,valid_days], datas[2][:,valid_days])

In [ ]:
assert n_stations_list[0] == n_stations_list[1] == n_stations_list[2] == n_stations_list[3]

In [ ]:
assert np.all(valid_days_list[0] == valid_days_list[0])
assert np.all(valid_days_list[0] == valid_days_list[1])
assert np.all(valid_days_list[0] == valid_days_list[2])
assert np.all(valid_days_list[0] == valid_days_list[3])
# assert np.all(valid_days_list[0] == valid_days_list[4])
# assert np.all(valid_days_list[0] == valid_days_list[5])
# assert np.all(valid_days_list[0] == valid_days_list[6])

In [ ]:
full_data = np.stack(datas_old, axis=2)

In [ ]:
full_data = full_data[:, valid_days_list[0], :]

n_days = full_data.shape[1]

assert n_stations == full_data.shape[0]

print(f'n_stations: {n_stations}, n_days: {n_days}')

In [ ]:
np.all(np.isnan(datas[0]).all(axis=(1)) == np.isnan(datas[1]).all(axis=(1)))

In [ ]:
keep = ~np.isnan(full_data).all(axis=1) # keep a station if we have at least one measurement in the period ot interest 
keep = np.all(keep, axis=1)

data = full_data[keep,:, :]
n_stations, n_days, _ = data.shape
print(f'n_stations: {n_stations}, n_days: {n_days}')

In [ ]:
keep_days = ~np.isnan(data).all(axis=0)
keep_days = np.all(keep_days, axis=1)
print(keep_days.shape)
print(data.shape)

In [ ]:
temp_data = full_data[:,:,1:3]
keep_temp = ~np.isnan(temp_data).all(axis=1)
keep_temp = np.all(keep_temp, axis=1)

temp_data = temp_data[keep_temp,:, :]
n_stations, n_days, _ = temp_data.shape
print(f'n_stations: {n_stations}, n_days: {n_days}')

In [ ]:
import cartopy.crs as ccrs

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

ax.set_global()
# ax.stock_img()
ax.coastlines()

plt.plot(lon[keep], lat[keep], 'or', marker='o', markerfacecolor='r', markersize=2)


In [ ]:
import cartopy.crs as ccrs

size = 40
fig = plt.figure(figsize=(2*size, 1*size))
for i in range(len(feature_names)):
    ax = fig.add_subplot(4, 1, i+1, projection=ccrs.PlateCarree())
    ax.set_global()
    ax.coastlines()
    ax.scatter(lon[keep], lat[keep], s=None, c=data[:,0,i], cmap=plt.get_cmap('RdYlGn'))
    ax.set_title(feature_names[i])

Seems to have outliers data. Can be sure to be correct (Tmax = -99 when Tmin = -8) or (Tmin = -72.8 for multiple Tmax)

In [ ]:
out = np.where(data<-70)
#print(data[out[:2]])
data[out] = np.nan

In [ ]:
plt.scatter(data[:,:,2], data[:,:,0])

In [ ]:
plt.scatter(data[:,:,2], data[:,:,1])

In [ ]:
plt.scatter(data[9,:,2], data[9,:,1])

In [ ]:
plt.scatter(data[:,:,1], data[:,:,0])

In [ ]:
plt.scatter(data[:,:,2], data[:,:,3])

In [ ]:
plt.scatter(data[:,:,0], data[:,:,3])

In [ ]:
plt.scatter(data[3,:,0], data[3,:,3])

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(data[:6,:,2].T, 'o')

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(data[0,:,0], 'o')
plt.figure(figsize=(20, 10))
plt.plot(data[0,:,2], 'o')
plt.figure(figsize=(20, 10))
plt.plot(data[1,:,2], 'o')

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(data[3,:,0], 'o')

In [ ]:
from pygsp import utils
from pygsp.graphs import Graph
from pygsp.graphs import NNGraph
class sphereGraph(NNGraph):
    def __init__(self, phi, theta, neighbors, rad=True, epsilon=True, **kwargs):
        if not rad:
            theta, phi = np.deg2rad(theta), np.deg2rad(phi)
        theta -= np.pi/2
        ct = np.cos(theta).flatten()
        st = np.sin(theta).flatten()
        cp = np.cos(phi).flatten()
        sp = np.sin(phi).flatten()
        x = st * cp
        y = st * sp
        z = ct
        self.coords = np.vstack([x, y, z]).T
        NNtype = 'radius' if epsilon else 'knn'
        plotting = {"limits": np.array([-1, 1, -1, 1, -1, 1])*0.5}
        self.n_vertices = len(self.coords)
        super(sphereGraph, self).__init__(self.coords, k=neighbors, NNtype=NNtype, center=False, rescale=False,
                                     plotting=plotting, **kwargs)

In [ ]:
g = sphereGraph(lon[keep], lat[keep], 100, rad=False)

In [ ]:
fig = plt.figure(figsize=(25,25))
axes = fig.add_subplot(111, projection='3d')
g.plot(vertex_size=10, edges=False, ax=axes)

In [ ]:
g.compute_laplacian("normalized")
g.compute_fourier_basis(recompute=True, n_eigenvectors=1000)
g.set_coordinates(g.U[:,1:4])
g.plot(vertex_size=10)

In [ ]:
plt.plot(g.e[:16], 'o')

In [ ]:
g_full = sphereGraph(lon, lat, 100, rad=False)

In [ ]:
fig = plt.figure(figsize=(25,25))
axes = fig.add_subplot(111, projection='3d')
g_full.plot(vertex_size=10, edges=False, ax=axes)

In [ ]:
g_full.compute_laplacian("normalized")
g_full.compute_fourier_basis(recompute=True, n_eigenvectors=500)
g_full.set_coordinates(g_full.U[:,1:4])
g_full.plot(vertex_size=10)

In [ ]:
plt.plot(g_full.e[:16], 'o')

## inference train

In [ ]:
import tensorflow as tf
params = {'L': [g.L.astype(np.float32)]*4,
          'p': [1,1,1,1],
          'F': [10, 20, 50, 1],
          'K': [5]*4,
          'batch_norm': [True]*4}
params['dir_name'] = 'GHCN_essai'
params['num_feat_in'] = 1
params['conv'] = 'chebyshev5'
params['pool'] = 'max'
params['activation'] = 'relu'
params['statistics'] = None#'mean'
params['regularization'] = 0
params['dropout'] = 1
params['num_epochs'] = 50  # Number of passes through the training data.
params['batch_size'] = 32
params['scheduler'] = lambda step: tf.train.exponential_decay(5e-1, step, decay_steps=5, decay_rate=1)
params['optimizer'] = lambda lr: tf.train.GradientDescentOptimizer(lr)
n_evaluations = 200
params['eval_frequency'] = int(params['num_epochs'] * n_days / params['batch_size'] / n_evaluations)
params['M'] = []
params['regression']=True
model = models.cgcnn(**params)

In [ ]:
from deepsphere.data import LabeledDatasetWithNoise, LabeledDataset

Temperature MAX from Temperature MIN

In [ ]:
dataset = full_data.transpose((1, 0, 2))
keepToo = ~np.isnan(dataset[:,:,1:3]).any(axis=0)
keepToo = keepToo.all(axis=1)
dataset_temp = dataset[:, keepToo, 1:3]

# remove outliers (remove outiler stations) (other option is to replace value by mean of knn)
out = np.where((dataset_temp<-60) + (dataset_temp>80))
dataset_temp = np.delete(dataset_temp, np.unique(out[1]), axis=1)
keepToo[np.argwhere(keepToo)[np.unique(out[1])]] = False

out = np.where(dataset_temp[:,:,1]==60)
dataset_temp = np.delete(dataset_temp, np.unique(out[1]), axis=1)
keepToo[np.argwhere(keepToo)[np.unique(out[1])]] = False

out = np.where((dataset_temp[:,:,0]==-35.6)*(dataset_temp[:,:,1]>0))
dataset_temp = np.delete(dataset_temp, np.unique(out[1]), axis=1)
keepToo[np.argwhere(keepToo)[np.unique(out[1])]] = False

# replace by means of neighbours (use graph)

In [ ]:
print(dataset[:,keepToo,1:3].shape)
print(np.isnan(dataset[:,keepToo,1:3]).all())

In [ ]:
import cartopy.crs as ccrs
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

ax.set_global()
# ax.stock_img()
ax.coastlines()

plt.plot(lon[keepToo], lat[keepToo], 'or', marker='o', markerfacecolor='r', markersize=2)

fig2 = plt.figure(figsize=(25,25))
axes = fig2.add_subplot(111, projection='3d')
gTemp = sphereGraph(lon[keepToo], lat[keepToo], 10, rad=False)
gTemp.plot(vertex_size=10, edges=True, ax=axes)
gTemp.compute_laplacian("normalized")

In [ ]:
## temp max from temp min

ratio = 0.7
n_days = dataset_temp.shape[0]
limit= int(ratio*n_days)

mean = dataset_temp.mean(axis=(0,1))[0]
std = dataset_temp.std(axis=(0,1))[0]

x_train = (dataset_temp[:limit,:,0] - mean) / std
labels_train = dataset_temp[:limit,:,1]
x_val = (dataset_temp[limit:,:,0] - mean) / std
labels_val = dataset_temp[limit:,:,1]


training = LabeledDataset(x_train, labels_train)
validation = LabeledDataset(x_val, labels_val)

In [ ]:
plt.scatter(x_val, labels_val)

In [ ]:
plt.scatter(x_train, labels_train)

Precipirtation from temperatures (MIN, MAX)

In [ ]:
dataset = full_data.transpose((1, 0, 2)).copy()
# remove outliers
### min temp is impossible
out = np.where(dataset[:,:,1]>100)
dataset[:,:,1][out] = np.nan

### min temp is always the same and max temp doesn't make sens
out = np.where(dataset[:,:,1]<-60)
dataset[:,:,1][out] = np.nan

### max temp is too big ofr the location and the min temp
out = np.where(dataset[:,:,2]>60)
dataset[:,:,2][out] = np.nan


## min is bigger than max
out = np.where(dataset[:,:,1]>dataset[:,:,2])
dataset[:,:,1:3][out] = np.nan

## dif between two days is bigger than 20° (what max is possible)
out = np.where(np.abs(np.diff(dataset[:,:,1:3], axis=0))>20)
dataset[:,:,1:3][out] = np.nan
dataset[:,:,1:3][out[0]+1,out[1], out[2]] = np.nan


keepToo = ~np.isnan(dataset[:,:,:3]).any(axis=0)
keepSuper = ((~np.isnan(dataset[:,:,:3])).sum(axis=0)>0.75*dataset.shape[0])
keepToo = keepToo.all(axis=1)
keepSuper = keepSuper.all(axis=1)
dataset_prec = dataset[:, keepToo, :3]

In [ ]:
plop = np.where(dataset[:,:,0]>3000)

In [ ]:
dataset[:,:,0][plop]

In [ ]:
np.isnan(dataset_prec).any()

In [ ]:
print("n stations minset = {}".format(keepToo.sum()))
print("n stations superset = {}".format(keepSuper.sum()))

In [ ]:
import cartopy.crs as ccrs

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

ax.set_global()
# ax.stock_img()
ax.coastlines()

plt.plot(lon[keepToo], lat[keepToo], 'or', marker='o', markerfacecolor='r', markersize=2)

In [ ]:
neighbour = 1
fig = plt.figure(figsize=(25,25))
axes = fig.add_subplot(111, projection='3d')
gPrec = sphereGraph(lon[keepToo], lat[keepToo], neighbour, rad=False)
gPrec.plot(vertex_size=10, edges=False, ax=axes)
gPrec.compute_laplacian("normalized")

In [ ]:
## precipitation from temp

ratio = 0.7
n_days = dataset_prec.shape[0]
limit= int(ratio*n_days)

mean = dataset_prec.mean(axis=(0,1))[1:3]
std = dataset_prec.std(axis=(0,1))[1:3]
# mean_label = dataset_prec.mean(axis=(0,1))[0]
# std_label = dataset_prec.std(axis=(0,1))[0]

x_train = (dataset_prec[:limit,:,1:3] - mean) / std
labels_train = dataset_prec[:limit,:,0]/10
x_val = (dataset_prec[limit:,:,1:3] - mean) / std
labels_val = dataset_prec[limit:,:,0]/10

# location of stations
coords_v = np.stack([lon[keepToo], lat[keepToo]], axis=-1)
coords_v = (coords_v-coords_v.mean(axis=0))/coords_v.std(axis=0)
# altitude of stations
alt_v = elev[keepToo]
alt_v = (alt_v-alt_v.mean())/alt_v.std()

x_train = np.dstack([x_train, np.repeat(coords_v[np.newaxis,:], x_train.shape[0], axis=0),
                     np.repeat(alt_v[np.newaxis,:], x_train.shape[0], axis=0),
                     np.repeat(w_days[:limit, np.newaxis], x_val.shape[1], axis=1)])
x_val = np.dstack([x_val, np.repeat(coords_v[np.newaxis,:], x_val.shape[0], axis=0),
                  np.repeat(alt_v[np.newaxis,:], x_val.shape[0], axis=0),
                  np.repeat(w_days[limit:, np.newaxis], x_val.shape[1], axis=1)])

training = LabeledDataset(x_train, labels_train)
validation = LabeledDataset(x_val, labels_val)

In [ ]:
plt.scatter(x_train[:,:,0], labels_train)

In [ ]:
plt.scatter(x_train[:,:,1], labels_train)

In [ ]:
plt.scatter(x_train[:,:,2], labels_train)

In [ ]:
plt.scatter(x_train[:,:,3], labels_train)

In [ ]:
plt.scatter(x_train[:,:,4], labels_train)

In [ ]:
plt.scatter(x_train[:,:,0], x_train[:,:,1])

In [ ]:
time = np.empty_like(x_train[:,:,0])
time[:,:] = np.arange(time.shape[0])[:,np.newaxis]

In [ ]:
plt.scatter(time, labels_train)

future temperature to find, regression

In [ ]:
valid_days[31+28::12*31]

In [ ]:
w_months = np.tile(np.repeat(np.arange(12), 31), years[-1]-years[0]+1)[valid_days]
w_days = np.tile(np.arange(365),years[-1]-years[0]+1)
w_days = np.insert(w_days, (3*365), 365)

In [ ]:
dataset = full_data.transpose((1, 0, 2)).copy()
# remove outliers
### min temp is impossible
out = np.where(dataset[:,:,1]>100)
dataset[:,:,1][out] = np.nan

### min temp is always the same and max temp doesn't make sens
out = np.where(dataset[:,:,1]<-60)
dataset[:,:,1][out] = np.nan

### max temp is too big ofr the location and the min temp
out = np.where(dataset[:,:,2]>=60)
dataset[:,:,2][out] = np.nan

## series of exact same values for the same location. seems strange
out = np.where((dataset[:,:,1]==-35.6)*(dataset[:,:,2]>0))
dataset[:,:,1][out] = np.nan

## min is bigger than max
out = np.where(dataset[:,:,1]>dataset[:,:,2])
dataset[:,:,1:3][out] = np.nan

## dif between two days is bigger than 20° (what max is possible)
out = np.where(np.abs(np.diff(dataset[:,:,1:3], axis=0))>20)
dataset[:,:,1:3][out] = np.nan
dataset[:,:,1:3][out[0]+1,out[1], out[2]] = np.nan


keepToo = ~np.isnan(dataset[:,:,1:3]).any(axis=0)
keepSuper = ((~np.isnan(dataset[:,:,1:3])).sum(axis=0)>0.75*dataset.shape[0])
keepToo = keepToo.all(axis=1)
keepSuper = keepSuper.all(axis=1)
dataset_temp_reg = dataset[:, keepToo, 1:3]

In [ ]:
print("n stations minset = {}".format(keepToo.sum()))
print("n stations superset = {}".format(keepSuper.sum()))

In [ ]:
import cartopy.crs as ccrs
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

ax.set_global()
# ax.stock_img()
ax.coastlines()

plt.plot(lon[keepToo], lat[keepToo], 'or', marker='o', markerfacecolor='r', markersize=2)

In [ ]:
neighbour = 10
fig = plt.figure(figsize=(25,25))
axes = fig.add_subplot(111, projection='3d')
gReg = sphereGraph(lon[keepToo], lat[keepToo], neighbour, rad=False)
gReg.plot(vertex_size=10, edges=False, ax=axes)
gReg.compute_laplacian("normalized")

In [ ]:
mask = ~np.isnan(dataset[:,keepSuper,1:3])
dataset_super = dataset[:, keepSuper, 1:3] # n_days graphs of superset
gRegSuper = sphereGraph(lon[keepSuper], lat[keepSuper], 100, rad=False)
gRegSuper.compute_laplacian("normalized")

In [ ]:
for day in range(dataset_super.shape[0]):
    for feat in range(dataset_super.shape[-1]):
        dataset_super[day,:,feat] = regression_tikhonov(gRegSuper, dataset_super[day,:,feat], mask[day,:,feat])

In [ ]:
mask.sum()

In [ ]:
np.isnan(dataset_super).sum()

In [ ]:
import numpy as np
from scipy import sparse

def regression_tikhonov(G, y, M, tau=0):
    r"""Solve a regression problem on graph via Tikhonov minimization.
    The function solves
    .. math:: \operatorname*{arg min}_x \| M x - y \|_2^2 + \tau \ x^T L x
    if :math:`\tau > 0`, and
    .. math:: \operatorname*{arg min}_x x^T L x \ \text{ s. t. } \ y = M x
    otherwise.
    Parameters
    ----------
    G : :class:`pygsp.graphs.Graph`
    y : array, length G.n_vertices
        Measurements.
    M : array of boolean, length G.n_vertices
        Masking vector.
    tau : float
        Regularization parameter.
    Returns
    -------
    x : array, length G.n_vertices
        Recovered values :math:`x`.
    Examples
    --------
    >>> from pygsp import graphs, filters, learning
    >>> import matplotlib.pyplot as plt
    >>>
    >>> G = graphs.Sensor(N=100, seed=42)
    >>> G.estimate_lmax()
    Create a smooth ground truth signal:
    >>> filt = lambda x: 1 / (1 + 10*x)
    >>> filt = filters.Filter(G, filt)
    >>> rs = np.random.RandomState(42)
    >>> signal = filt.analyze(rs.normal(size=G.n_vertices))
    Construct a measurement signal from a binary mask:
    >>> mask = rs.uniform(0, 1, G.n_vertices) > 0.5
    >>> measures = signal.copy()
    >>> measures[~mask] = np.nan
    Solve the regression problem by reconstructing the signal:
    >>> recovery = learning.regression_tikhonov(G, measures, mask, tau=0)
    Plot the results:
    >>> fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(10, 3))
    >>> limits = [signal.min(), signal.max()]
    >>> _ = G.plot_signal(signal, ax=ax1, limits=limits, title='Ground truth')
    >>> _ = G.plot_signal(measures, ax=ax2, limits=limits, title='Measures')
    >>> _ = G.plot_signal(recovery, ax=ax3, limits=limits, title='Recovery')
    >>> _ = fig.tight_layout()
    """

    if tau > 0:

        y[M == False] = 0

        if sparse.issparse(G.L):

            def Op(x):
                return (M * x.T).T + tau * (G.L.dot(x))

            LinearOp = sparse.linalg.LinearOperator([G.N, G.N], Op)

            if y.ndim > 1:
                sol = np.empty(shape=y.shape)
                res = np.empty(shape=y.shape[1])
                for i in range(y.shape[1]):
                    sol[:, i], res[i] = sparse.linalg.cg(
                        LinearOp, y[:, i])
            else:
                sol, res = sparse.linalg.cg(LinearOp, y)

            # TODO: do something with the residual...
            return sol

        else:

            # Creating this matrix may be problematic in term of memory.
            # Consider using an operator instead...
            if type(G.L).__module__ == np.__name__:
                LinearOp = np.diag(M*1) + tau * G.L
            return np.linalg.solve(LinearOp, M * y)

    else:

        if np.prod(M.shape) != G.n_vertices:
            raise ValueError("M should be of size [G.n_vertices,]")

        indl = M
        indu = (M == False)

        Luu = G.L[indu, :][:, indu]
        Wul = - G.L[indu, :][:, indl]

        if sparse.issparse(G.L):
            sol_part = sparse.linalg.spsolve(Luu, Wul.dot(y[indl]))
        else:
            sol_part = np.linalg.solve(Luu, np.matmul(Wul, y[indl]))

        sol = y.copy()
        sol[indu] = sol_part

        return sol

In [ ]:
coords_v = (coords_v-coords_v.mean(axis=0))/coords_v.std(axis=0)
plt.plot(coords_v[:,0], coords_v[:,1], 'o', markersize=1)

In [ ]:
coords_v[0]

In [ ]:
## temp future (regression)
days_pred = 5

n_days, n_stations, n_feature= dataset_temp_reg.shape

dataset_x = np.vstack([np.roll(dataset_temp_reg, -i, axis=0) for i in range(days_pred)])
dataset_x = dataset_x.reshape(days_pred, n_days, n_stations, n_feature).transpose((1,2,3,0))

# mean_x = dataset_x.mean(axis=(0,1))
# std_x = dataset_x.std(axis=(0,1))

# month_x = np.vstack([np.roll(w_months, -i, axis=0) for i in range(days_pred)])
# month_x = month_x.reshape(days_pred, n_days).transpose()

days_x = np.vstack([np.roll(w_days, -i, axis=0) for i in range(days_pred)])
days_x = days_x.reshape(days_pred, n_days).transpose()

x_train = dataset_x[:n_days-days_pred,:,0,:]
labels_train = dataset_temp_reg[days_pred:,:,0]
x_val = dataset_x[:n_days-days_pred,:,1,:]
labels_val = dataset_temp_reg[days_pred:,:,1]

# x_train = (x_train-mean_x[0])/std_x[0]
# x_val = (x_val-mean_x[1])/std_x[1]

# location of stations
coords_v = np.stack([lon[keepToo], lat[keepToo]], axis=-1)
coords_v = (coords_v-coords_v.mean(axis=0))/coords_v.std(axis=0)
# altitude of stations
alt_v = elev[keepToo]
alt_v = (alt_v-alt_v.mean())/alt_v.std()

# can include information on which period we are? month?
x_train = np.dstack([x_train, 
#                     np.broadcast_to(month_x[:n_days-days_pred,np.newaxis, :], x_train.shape),
                     np.repeat(coords_v[np.newaxis,:], x_train.shape[0],axis=0),
                     np.repeat(alt_v[np.newaxis,:], x_train.shape[0],axis=0),
                     np.broadcast_to(days_x[:n_days-days_pred,np.newaxis, :], x_train.shape)])
x_val = np.dstack([x_val, 
#                   np.broadcast_to(month_x[:n_days-days_pred,np.newaxis, :], x_val.shape), 
                   np.repeat(coords_v[np.newaxis,:], x_train.shape[0],axis=0),
                   np.repeat(alt_v[np.newaxis,:], x_train.shape[0],axis=0),
                   np.broadcast_to(days_x[:n_days-days_pred,np.newaxis, :], x_val.shape)])

training = LabeledDataset(x_train, labels_train)
validation = LabeledDataset(x_val, labels_val)

In [ ]:
plt.scatter(dataset_temp_reg[:,:,0], dataset_temp_reg[:,:,1])

In [ ]:
timereg = np.empty_like(dataset_temp_reg[:,:,0])
timereg[:,:] = np.arange(timereg.shape[0])[:,np.newaxis]
plt.scatter(timereg, dataset_temp_reg[:,:,0])

In [ ]:
placereg = np.empty_like(dataset_temp_reg[:,:,0])
placereg[:,:] = np.arange(placereg.shape[1])
plt.scatter(placereg, dataset_temp_reg[:,:,0])

In [ ]:
## is it possible to have a difference of 40 degrees between 1 day?  (must fix to +- 20 degrees)
plt.scatter(x_train[:,:,days_pred-1], labels_train)

# Misc

In [ ]:
mat_nan = np.isnan(full_data[:,:,0])
fig=plt.figure(figsize=(55,100))
ax = fig.add_subplot(421)
ax.spy(mat_nan, aspect='auto')
keep_ = ~np.isnan(full_data[:,:,0]).all(axis=1)
mat_nan = np.isnan(full_data[keep_,:,0])
ax = fig.add_subplot(422)
ax.spy(mat_nan, aspect='auto')

mat_nan = np.isnan(full_data[:,:,1])
ax = fig.add_subplot(423)
ax.spy(mat_nan, aspect='auto')
keep_ = ~np.isnan(full_data[:,:,1]).all(axis=1)
mat_nan = np.isnan(full_data[keep_,:,1])
ax = fig.add_subplot(424)
ax.spy(mat_nan, aspect='auto')

mat_nan = np.isnan(full_data[:,:,2])
ax = fig.add_subplot(425)
ax.spy(mat_nan, aspect='auto')
keep_ = ~np.isnan(full_data[:,:,2]).all(axis=1)
mat_nan = np.isnan(full_data[keep_,:,2])
ax = fig.add_subplot(426)
ax.spy(mat_nan, aspect='auto')

mat_nan = np.isnan(full_data[:,:,3])
ax = fig.add_subplot(427)
ax.spy(mat_nan, aspect='auto')
keep_ = ~np.isnan(full_data[:,:,3]).all(axis=1)
mat_nan = np.isnan(full_data[keep_,:,3])
ax = fig.add_subplot(428)
ax.spy(mat_nan, aspect='auto')

In [ ]:
#feature_name = 'TMAX'
# feature_names
filenames = []
datas = []
valid_days_list = []
n_stations_list = []
n_years = len(years2)
for feature_name in feature_names:
    filenames.append('data_{:4d}-{:4d}_{}.npz'.format(years[0], years2[-1], feature_name))
    print(f'- Checking if file {filenames[-1]} exists..')

    # only recompute if necessary
    if not os.path.isfile(datapath+filenames[-1]):

        print('- The file is not there. Parsing everything from raw. This will take a while.')
        # Load the station measurements into a year-list of dataframes
        df_years = []

        for yearIdx,year in enumerate(years):

            filename_year = 'raw/{:4}.csv.gz'.format(year)
            print(' - parsing *{}*'.format(filename_year))

            df = pd.read_csv(filename_year, names=['id_ghcn', 'date', 'type', 'value', '?0', '?1', '?2', '?3'], \
                             nrows=None, usecols=[0,1,2,3])

            # create a new column with the id_local
            id_local = [ghcn_to_local.get(id_g) for id_g in df["id_ghcn"].values]
            id_local = [-1 if v is None else v for v in id_local]
            id_local = np.array(id_local).astype(np.int)

            df = df.assign(id_local=pd.Series(id_local, index=df.index).values)

            # remove measurement of stations with unknown id_local
            df = df[df.id_local != -1] 

            df_years.append(df)

        del df, id_local
        print('done!')

        # Construct one array per feature and save it to disk

        # indicate for which days we have measurements (this also deals with months of different lengths)
        valid_days = np.zeros((n_years, 12, 31), dtype=np.bool)

        for _, name in enumerate(feature_names):

            print(f' - Looking at {name}')

            data = np.zeros((n_stations, n_years, 12, 31), dtype=np.float) * np.nan

            for yearIdx,year in enumerate(years):

                df = df_years[yearIdx]
                idf = df.loc[df['type'] == name]

                print(f'  - year {year}')

                # remove measurement of stations with unknown id_local
                idf = idf[idf.id_local != -1] 

                for monthIdx,month in enumerate(range(1,12+1)): 
                    for dayIdx,day in enumerate(range(1,31+1)):        

                        date = int('{:4d}{:02d}{:02d}'.format(year,month,day))
                        jdf = idf.loc[idf['date'] == date]

                        # sort data according to the id_local 
                        jdf.set_index('id_local', inplace=True)
                        jdf = jdf.sort_index()

                        index = jdf.index.values
                        values = jdf['value'].values.astype(np.float)

                        if len(index) != 0: 
                            data[index,yearIdx,monthIdx,dayIdx] = values
                            valid_days[yearIdx,monthIdx,dayIdx] = True

            print('  - saving to disk')
            np.savez_compressed('processed/data_{:4d}-{:4d}_{}.npz'.format(years[0], years[-1], name), data=data, valid_days=valid_days)

            del index, values, df, idf, jdf    

    else:    
        print('- Loading data from disk..')

        data_file = np.load(datapath+filenames[-1])
        data, valid_days = data_file['data'], data_file['valid_days']        
        n_stations = data.shape[0]
        print(f'- {n_stations} stations loaded.')
        data = data.reshape((n_stations, n_years*12*31))
        if feature_name == 'TMIN' or feature_name == 'TMAX':
            data = data.astype(np.float)
            data /= 10
        datas.append(data)
        valid_days = np.squeeze(valid_days.reshape(n_years*12*31)).astype(np.bool)
        valid_days_list.append(valid_days)
        n_stations_list.append(n_stations)

In [ ]:
full_data2 = np.stack(datas, axis=2)
full_data2 = full_data2[:, valid_days_list[0], :]

n_days = full_data2.shape[1]

assert n_stations == full_data2.shape[0]

print(f'n_stations: {n_stations}, n_days: {n_days}')

In [ ]:
mat_nan = np.isnan(full_data2[:,:,0])
fig=plt.figure(figsize=(55,100))
ax = fig.add_subplot(421)
ax.spy(mat_nan, aspect='auto')
keep_ = ~np.isnan(full_data2[:,:,0]).all(axis=1)
mat_nan = np.isnan(full_data2[keep_,:,0])
ax = fig.add_subplot(422)
ax.spy(mat_nan, aspect='auto')

mat_nan = np.isnan(full_data2[:,:,1])
ax = fig.add_subplot(423)
ax.spy(mat_nan, aspect='auto')
keep_ = ~np.isnan(full_data2[:,:,1]).all(axis=1)
mat_nan = np.isnan(full_data2[keep_,:,1])
ax = fig.add_subplot(424)
ax.spy(mat_nan, aspect='auto')

mat_nan = np.isnan(full_data2[:,:,2])
ax = fig.add_subplot(425)
ax.spy(mat_nan, aspect='auto')
keep_ = ~np.isnan(full_data2[:,:,2]).all(axis=1)
mat_nan = np.isnan(full_data2[keep_,:,2])
ax = fig.add_subplot(426)
ax.spy(mat_nan, aspect='auto')

mat_nan = np.isnan(full_data2[:,:,3])
ax = fig.add_subplot(427)
ax.spy(mat_nan, aspect='auto')
keep_ = ~np.isnan(full_data2[:,:,3]).all(axis=1)
mat_nan = np.isnan(full_data2[keep_,:,3])
ax = fig.add_subplot(428)
ax.spy(mat_nan, aspect='auto')

In [ ]:
print(sum(~np.isnan(full_data2[:,:,0]).any(axis=1)))
print(sum(~np.isnan(full_data2[:,:,1]).any(axis=1)))
print(sum(~np.isnan(full_data2[:,:,2]).any(axis=1)))
print(sum(~np.isnan(full_data2[:,:,3]).any(axis=1)))

In [ ]:
set1=set(np.where(~np.isnan(full_data2[:,:,0]).any(axis=1))[0])
set2=set(np.where(~np.isnan(full_data2[:,:,1]).any(axis=1))[0])
set3=set(np.where(~np.isnan(full_data2[:,:,2]).any(axis=1))[0])
set4=set(np.where(~np.isnan(full_data2[:,:,3]).any(axis=1))[0])
print(len(set1.intersection(set2, set3, set4)))
print(len(set2.intersection(set3)))

In [ ]:
(~np.isnan(full_data[keep_,:,3][:,ploup[0]])).sum(axis=1)

In [ ]:
plop = np.where((~np.isnan(full_data[keep_,:,3][:,ploup[0]])).sum(axis=1)>69)  # stations

In [ ]:
ploup = np.where((~np.isnan(full_data[:,:,3])).sum(axis=0)>15000)  # days

In [ ]:
mat_nan = np.isnan(full_data[keep_,:,3][plop[0],:])
fig=plt.figure(figsize=(25,25))
ax = fig.add_subplot(111)
ax.spy(mat_nan, aspect='auto')

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

ax.set_global()
# ax.stock_img()
ax.coastlines()

plt.plot(lon[plop], lat[plop], 'or', marker='o', markerfacecolor='r', markersize=2)